# Pull all features for an area

In this notebook, I'm developing and testing code that pulls and standardizes all 
of the map features that we are interested in, for a box of a given size.

In [ ]:
import numpy as np
import pandas as pd
import pyproj
import shapely
import osmnx

import plotly
from plotly.subplots import make_subplots
from plotly.graph_objects import Scatter

from geo_encodings import draw_shape

import utils


## Parameters

In [ ]:
# AOIs are squares of this dimension, in meters.
aoi_size = 1000

# For testing
lat, lon = 42.981163, -70.946524 # Exeter NH
# lat, lon = 43.077132, -70.757544 # Portsmouth NH



## Preliminaries

In [ ]:
# Define a local map projection

center_lat = lat
center_lon = lon

proj_def = f"""
+proj=tmerc +lat_0={center_lat} +lon_0={center_lon} 
+k=1.0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs
"""

ltm_crs = pyproj.CRS.from_proj4(proj_def)
wgs84_crs = pyproj.CRS.from_epsg(4326)
proj_forward = pyproj.Transformer.from_crs(wgs84_crs, ltm_crs, always_xy=True).transform
proj_inverse = pyproj.Transformer.from_crs(ltm_crs, wgs84_crs, always_xy=True).transform



In [ ]:
# Use that projection to define lon/lat nbounds. Make sure the bounds go 
# a little farther out than necessary to avoid edge artifacts from map projections.
dd = aoi_size * 0.6

lon0, lat0 = proj_inverse(-dd, -dd)
lon1, lat1 = proj_inverse(+dd, +dd)
print(lon0, lat0, lon1, lat1)

In [ ]:
# Define the boundng box to be used to query OSM.
query_bounds = [lon0, lat0, lon1, lat1]
center_lon = (lon0 + lon1) / 2
center_lat = (lat0 + lat1) / 2

In [ ]:
# tags = {
#     'landuse': True,
#     'place': True,
#     'highway': True,
#     'railway': True,
#     'aeroway': True,
#     'bridge': True,
#     'tunnel': True,
#     'power': True,
#     'natural': True,
#     'waterway': True,
#     'landcover': True,
#     'building': True,
#     'amenity': True,
#     'shop': True,
#     'leisure': True
# }
# features = osmnx.features.features_from_bbox(query_bounds, tags=tags).reset_index()
# print('%d features' % len(features))

In [ ]:
keepers = []
missed = []

# for key in ['highway', 'waterway', 'landuse', 'railway', 'bridge', 'tunnel', 'building', 'amenity', 'shop', 'leisure']:
for key in ['highway', 'waterway', 'landuse', 'railway', 'bridge', 'tunnel', 'building', 'amenity', 'shop',]:
    print(key)
    
    query_tags = {key: True}
    try:
        features = osmnx.features.features_from_bbox(query_bounds, tags=query_tags).reset_index()
    except:
        print('==> query failed')
        continue
    
    for feature in features.to_dict('records'):
        
        geomxy = shapely.ops.transform(proj_forward, feature['geometry'])
        gtype = geomxy.geom_type

        value = 'unknown'
        for k in query_tags.keys():
            if str(feature[k]) != 'nan':
                value = str(feature[k])
                
        # Apply reformatting rules
        matched = False
        for rule in utils.rules[key]:
            if gtype == rule['gtype']:
    
                # Check any keys for this feature class.
                all_match = True
                for rule_key in rule['keys']:
                    rule_values = rule['keys'][rule_key]
                    if rule_key not in features or value not in rule_values:
                        all_match = False
                if all_match:
                    keepers.append({
                        'category': rule['category'],
                        'label': rule['label'],
                        'gtype': gtype,
                        'geom': geomxy,
                    })
                    matched = True
                if not matched:
                    missed.append({
                        'category': key,
                        'gtype': gtype
                    })
            if matched:
                break


In [ ]:
features.iloc[1].to_dict()

In [ ]:
pd.DataFrame(keepers)[['category', 'label', 'gtype']].value_counts().sort_index()